In [47]:
import yt
from yt.units import dimensions
import numpy as np
import unyt
import scipy.interpolate
import matplotlib.pyplot as plt

def _abs_z(field, data):
    return np.abs(data["gas", "z"])

yt.add_field(
    name=("gas", "abs_z"),
    function=_abs_z,
    sampling_type="local",
    units="auto",
    dimensions=dimensions.length,
)

In [52]:
root_dir = "/scratch/cvz/bwibking/precipitator/solenoidal/tc_tff_2.0/output_tcorr_500_eff_0.97/snapshots/"
filenames = [(root_dir + "parthenon.prim." + str(i).zfill(5) + ".phdf", "parthenon.prim." + str(i).zfill(5)) for i in range(40)]

for filename, filename_base in filenames:
    ds = yt.load(filename)
    source = ds.all_data()

    # compute entropy histogram
    K_bins = np.logspace(-4.5, -3.5, 20)
    z_bins = np.logspace(np.log10(10.), 2.0, 20) * 3.086e18 * 1.0e3
    histogram = yt.data_objects.profiles.create_profile(source,
        [("gas", "abs_z"), ("parthenon", "entropy")],
        [("gas", "cell_mass")]
        , override_bins={("parthenon", "entropy"): K_bins, ("gas", "abs_z"): z_bins})

    bins_x = np.log10(histogram.x / 3.086e18 / 1.0e3) # abs_z
    bins_y = np.log10(histogram.y) # entropy
    hist = histogram["gas", "cell_mass"] / 1.99e33  # profile field

    lb = np.zeros(len(bins_x))
    ub = np.zeros(len(bins_x))
    median = np.zeros(len(bins_x))
    for i in range(len(bins_x)):
        cdf = np.concatenate([np.array([0]), np.cumsum(hist[i,:])])
        cdf /= cdf[-1]
        interp = scipy.interpolate.interp1d(cdf, np.log10(K_bins), kind='linear')
        median[i] = interp(0.5)
        lb[i] = interp(0.16)
        ub[i] = interp(0.84)
    
    plt.clf()
    plt.plot(bins_x, median, color='black', label='median $\log K$')
    plt.fill_between(bins_x, lb, ub, alpha=0.5, label=r'1$\sigma$ region')
    plt.ylim(-4.5, -3.5)
    plt.xlabel(r'$\log |z|$ (kpc)')
    plt.ylabel(r'$\log K$')
    plt.legend(loc='best')
    plt.savefig(filename_base + ".png")

FileNotFoundError: No such file or directory: '/scratch/cvz/bwibking/precipitator/solenoidal/tc_tff_2.0/output_tcorr_500_eff_0.97/snapshots/parthenon.prim.00000.phdf'.